In [ ]:
import os
import sys
import subprocess

sys.path.insert(0, "/Users/connorparish/code/hindsight")

from hindsight_server.db import HindsightDB
from hindsight_server.utils import add_datetimes
from hindsight_server.config import VIDEO_FILES_DIR

In [ ]:
db = HindsightDB()

In [ ]:
video_chunks = db.get_video_chunks()

In [ ]:
frames = db.get_frames(impute_applications=False)

In [ ]:
frames = frames.loc[frames['video_chunk_id'].isnull()]

In [ ]:
frames = frames.loc[~frames['application'].isin(["backCamera", "frontCamera"])]

In [ ]:
frames = add_datetimes(frames)

In [ ]:
def get_video_path(row):
    formatted_date = row['datetime_utc'].strftime("%Y%m%d")
    return os.path.join(VIDEO_FILES_DIR, f"{row['application_org']}_{formatted_date}_og.mp4")
frames["video_path"] = frames.apply(lambda row: get_video_path(row), axis=1)

In [ ]:
frames = frames.sort_values(by="timestamp", ascending=True)

In [ ]:
def compress_images_to_mp4(image_files, output_file, fps=0.5):
    """
    Compress a list of .jpg files into an MP4 video using the H.264 codec.
    
    Args:
        image_files (list): List of paths to .jpg image files.
        output_file (str): Path to the output .mp4 file.
        fps (int): Frames per second for the video.
    """
    if not image_files:
        raise ValueError("The image_files list is empty.")
    
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a temporary file list for ffmpeg
    temp_file_list = "temp_file_list.txt"
    with open(temp_file_list, "w") as file:
        for image in image_files:
            file.write(f"file '{os.path.abspath(image)}'\n")

    success = False
    try:
        # Construct the ffmpeg command
        command = [
            "ffmpeg",
            "-y",  # Overwrite output file if it exists
            "-f", "concat",  # Input format
            "-safe", "0",  # Enable safe file paths
            "-r", str(fps),  # Frames per second
            "-i", temp_file_list,  # Input file list
            "-c:v", "libx264",  # Video codec
            "-pix_fmt", "yuv420p",  # Pixel format for compatibility
            output_file  # Output file
        ]

        # Run the ffmpeg command
        subprocess.run(command, check=True)
        print(f"Video successfully created at: {output_file}")
        if os.path.exists(output_file) and os.path.getsize(output_file) > 1000:
            success = True
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while creating the video: {e}")
        success = False
    finally:
        # Clean up the temporary file list
        if os.path.exists(temp_file_list):
            os.remove(temp_file_list)
        return success

In [ ]:
vc_count = 0
successful_list = list()
for video_path in frames.video_path.unique():
    if os.path.exists(video_path):
        print(video_path, "already exists")
        continue
    # if vc_count > 1000:
    #     break
    video_path_df = frames.loc[frames['video_path'] == video_path]
    video_path_df = video_path_df.sort_values(by="timestamp", ascending=True)
    print(len(video_path_df))
    success = compress_images_to_mp4(list(video_path_df['path']), video_path)
    successful_list.append(success)
    if success:
        video_chunk_id = db.insert_video_chunk(video_path)
        db.update_video_chunk_info(video_chunk_id=video_chunk_id, frame_ids=list(video_path_df['id']))
        for f in list(video_path_df['path']):
            if os.path.exists(f):
                os.remove(f)
                print(f"Deleted {f}")
        vc_count += 1